In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from qiskit.circuit.library.standard_gates import RXGate, RZGate, RYGate, CXGate, CZGate, SGate, HGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

import numpy as np

In [3]:
def get_linear_entangelment_ansatz(num_of_qubits, thetas, input_state, circuit_depth=3):
    quantum_register = QuantumRegister(num_of_qubits, name="qubit")
    quantum_circuit = QuantumCircuit(quantum_register)
    quantum_circuit.initialize(input_state)

    for iteration in range(circuit_depth):
        for qubit_index in range(num_of_qubits):
            RY_theta_index = iteration*2*num_of_qubits + qubit_index
            RZ_theta_index = RY_theta_index + num_of_qubits
            
            quantum_circuit.append(RYGate(thetas[RY_theta_index]), [quantum_register[qubit_index]])
            quantum_circuit.append(RZGate(thetas[RZ_theta_index]), [quantum_register[qubit_index]])
            
        for qubit_index in range(num_of_qubits - 1):
            quantum_circuit.append(CXGate(), [quantum_register[qubit_index], quantum_register[qubit_index + 1]])

    for qubit_index in range(num_of_qubits):
            RY_theta_index = 2*num_of_qubits*circuit_depth + qubit_index
            RZ_theta_index = RY_theta_index + num_of_qubits
            quantum_circuit.append(RYGate(thetas[RY_theta_index]), [quantum_register[qubit_index]])
            quantum_circuit.append(RZGate(thetas[RZ_theta_index]), [quantum_register[qubit_index]])

    return quantum_circuit

In [4]:
def get_full_entangelment_ansatz(num_of_qubits, thetas, input_state, circuit_depth=3):
    quantum_register = QuantumRegister(num_of_qubits, name="qubit")
    quantum_circuit = QuantumCircuit(quantum_register)
    quantum_circuit.initialize(input_state)

    for iteration in range(circuit_depth):
        for qubit_index in range(num_of_qubits):
            RY_theta_index = iteration*2*num_of_qubits + qubit_index
            RZ_theta_index = RY_theta_index + num_of_qubits
            
            quantum_circuit.append(RYGate(thetas[RY_theta_index]), [quantum_register[qubit_index]])
            quantum_circuit.append(RZGate(thetas[RZ_theta_index]), [quantum_register[qubit_index]])
        for qubit_index in range(num_of_qubits - 1):
            for target_qubit_index in range(qubit_index + 1, num_of_qubits):
                quantum_circuit.append(CXGate(), [quantum_register[qubit_index], quantum_register[target_qubit_index]])

    for qubit_index in range(num_of_qubits):
            RY_theta_index = 2*num_of_qubits*circuit_depth + qubit_index
            RZ_theta_index = RY_theta_index + num_of_qubits
            
            quantum_circuit.append(RYGate(thetas[RY_theta_index]), [quantum_register[qubit_index]])
            quantum_circuit.append(RZGate(thetas[RZ_theta_index]), [quantum_register[qubit_index]])

    return quantum_circuit

## Linear entangelment ansatz

In [5]:
initial_thetas = np.random.uniform(low=0, high=360, size=32)
initial_eigenvector = np.identity(16)[0]
print(get_linear_entangelment_ansatz(4, initial_thetas, 3))

         ┌────────────────┐┌────────────┐ ┌─────────┐       ┌────────────┐»
qubit_0: ┤0               ├┤ Ry(224.73) ├─┤ Rz(203) ├────■──┤ Ry(23.197) ├»
         │                │├───────────┬┘┌┴─────────┴─┐┌─┴─┐└────────────┘»
qubit_1: ┤1               ├┤ Ry(324.1) ├─┤ Rz(173.94) ├┤ X ├──────■───────»
         │  Initialize(3) │├───────────┴┐├────────────┤└───┘    ┌─┴─┐     »
qubit_2: ┤2               ├┤ Ry(122.99) ├┤ Rz(207.45) ├─────────┤ X ├─────»
         │                │├───────────┬┘├────────────┤         └───┘     »
qubit_3: ┤3               ├┤ Ry(24.76) ├─┤ Rz(279.09) ├───────────────────»
         └────────────────┘└───────────┘ └────────────┘                   »
«         ┌────────────┐                            ┌────────────┐»
«qubit_0: ┤ Rz(118.29) ├────────────────────■───────┤ Ry(111.57) ├»
«         ├───────────┬┘┌────────────┐    ┌─┴─┐     └────────────┘»
«qubit_1: ┤ Ry(205.1) ├─┤ Rz(318.52) ├────┤ X ├───────────■───────»
«         └───────────┘ ├────────────┤┌───┴─

In [6]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=24)
initial_eigenvector = np.identity(8)[0]
print(get_linear_entangelment_ansatz(3, initial_thetas, 3))

         ┌────────────────┐ ┌────────────┐ ┌────────────┐     ┌────────────┐»
qubit_0: ┤0               ├─┤ Ry(4.7365) ├─┤ Rz(2.6133) ├──■──┤ Ry(3.0644) ├»
         │                │┌┴────────────┤┌┴────────────┤┌─┴─┐└────────────┘»
qubit_1: ┤1 Initialize(3) ├┤ Ry(0.37922) ├┤ Rz(0.97179) ├┤ X ├──────■───────»
         │                │└┬────────────┤└┬────────────┤└───┘    ┌─┴─┐     »
qubit_2: ┤2               ├─┤ Ry(4.7539) ├─┤ Rz(2.6994) ├─────────┤ X ├─────»
         └────────────────┘ └────────────┘ └────────────┘         └───┘     »
«         ┌────────────┐                   ┌────────────┐┌────────────┐»
«qubit_0: ┤ Rz(5.7441) ├────────────────■──┤ Ry(4.0801) ├┤ Rz(3.0293) ├»
«         ├────────────┤┌────────────┐┌─┴─┐└────────────┘├────────────┤»
«qubit_1: ┤ Ry(4.2136) ├┤ Rz(3.5615) ├┤ X ├──────■───────┤ Ry(5.7047) ├»
«         ├────────────┤├────────────┤└───┘    ┌─┴─┐     ├────────────┤»
«qubit_2: ┤ Ry(3.9104) ├┤ Rz(5.2879) ├─────────┤ X ├─────┤ Ry(5.2453) ├»
«         └─────

In [7]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=16)
initial_eigenvector = np.identity(4)[0]
print(get_linear_entangelment_ansatz(2, initial_thetas, 3))

         ┌────────────────┐ ┌────────────┐┌────────────┐     ┌─────────────┐»
qubit_0: ┤0               ├─┤ Ry(3.2007) ├┤ Rz(6.2109) ├──■──┤ Ry(0.98935) ├»
         │  Initialize(3) │┌┴────────────┤├────────────┤┌─┴─┐└┬────────────┤»
qubit_1: ┤1               ├┤ Ry(0.47116) ├┤ Rz(1.4293) ├┤ X ├─┤ Ry(2.1767) ├»
         └────────────────┘└─────────────┘└────────────┘└───┘ └────────────┘»
«         ┌────────────┐     ┌────────────┐ ┌────────────┐     ┌────────────┐»
«qubit_0: ┤ Rz(2.3647) ├──■──┤ Ry(6.1192) ├─┤ Rz(4.7531) ├──■──┤ Ry(4.3212) ├»
«         ├────────────┤┌─┴─┐├────────────┤┌┴────────────┤┌─┴─┐├────────────┤»
«qubit_1: ┤ Rz(1.7455) ├┤ X ├┤ Ry(1.9402) ├┤ Rz(0.15783) ├┤ X ├┤ Ry(6.0535) ├»
«         └────────────┘└───┘└────────────┘└─────────────┘└───┘└────────────┘»
«          ┌────────────┐
«qubit_0: ─┤ Rz(1.6645) ├
«         ┌┴────────────┤
«qubit_1: ┤ Rz(0.42372) ├
«         └─────────────┘


## Full entangelment ansatz

In [8]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=32)
initial_eigenvector = np.identity(16)[0]
print(get_full_entangelment_ansatz(4, initial_thetas, 3))

         ┌────────────────┐ ┌────────────┐┌────────────┐               »
qubit_0: ┤0               ├─┤ Ry(4.7757) ├┤ Rz(5.5463) ├──■────■────■──»
         │                │ ├────────────┤├────────────┤┌─┴─┐  │    │  »
qubit_1: ┤1               ├─┤ Ry(4.8503) ├┤ Rz(0.7906) ├┤ X ├──┼────┼──»
         │  Initialize(3) │┌┴────────────┤├───────────┬┘└───┘┌─┴─┐  │  »
qubit_2: ┤2               ├┤ Ry(0.96564) ├┤ Rz(1.683) ├──────┤ X ├──┼──»
         │                │└┬────────────┤├───────────┤      └───┘┌─┴─┐»
qubit_3: ┤3               ├─┤ Ry(1.5927) ├┤ Rz(1.551) ├───────────┤ X ├»
         └────────────────┘ └────────────┘└───────────┘           └───┘»
«         ┌────────────┐┌─────────────┐                             »
«qubit_0: ┤ Ry(2.6514) ├┤ Rz(0.86981) ├─────────────────────────────»
«         └────────────┘└─────────────┘┌────────────┐┌─────────────┐»
«qubit_1: ──────■──────────────■───────┤ Ry(5.7825) ├┤ Rz(0.87181) ├»
«             ┌─┴─┐            │       └────────────┘└┬────────

In [9]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=24)
initial_eigenvector = np.identity(8)[0]
print(get_full_entangelment_ansatz(3, initial_thetas, 3))

         ┌────────────────┐┌─────────────┐┌────────────┐          »
qubit_0: ┤0               ├┤ Ry(0.38159) ├┤ Rz(0.2804) ├──■────■──»
         │                │└┬────────────┤├────────────┤┌─┴─┐  │  »
qubit_1: ┤1 Initialize(3) ├─┤ Ry(3.8785) ├┤ Rz(5.0571) ├┤ X ├──┼──»
         │                │ ├───────────┬┘├────────────┤└───┘┌─┴─┐»
qubit_2: ┤2               ├─┤ Ry(3.393) ├─┤ Rz(3.5364) ├─────┤ X ├»
         └────────────────┘ └───────────┘ └────────────┘     └───┘»
«         ┌────────────┐ ┌────────────┐                         ┌────────────┐»
«qubit_0: ┤ Ry(1.5353) ├─┤ Rz(5.5166) ├─────────────────■────■──┤ Ry(1.2206) ├»
«         └────────────┘┌┴────────────┤┌─────────────┐┌─┴─┐  │  └────────────┘»
«qubit_1: ──────■───────┤ Ry(0.78391) ├┤ Rz(0.71301) ├┤ X ├──┼────────■───────»
«             ┌─┴─┐     └┬───────────┬┘├─────────────┤└───┘┌─┴─┐    ┌─┴─┐     »
«qubit_2: ────┤ X ├──────┤ Ry(4.806) ├─┤ Rz(0.48249) ├─────┤ X ├────┤ X ├─────»
«             └───┘      └───────────┘ └────

In [10]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=16)
initial_eigenvector = np.identity(4)[0]
print(get_full_entangelment_ansatz(2, initial_thetas, 3))

         ┌────────────────┐┌────────────┐┌─────────────┐     ┌────────────┐»
qubit_0: ┤0               ├┤ Ry(3.0783) ├┤ Rz(0.25354) ├──■──┤ Ry(5.1894) ├»
         │  Initialize(3) │├────────────┤└┬────────────┤┌─┴─┐├────────────┤»
qubit_1: ┤1               ├┤ Ry(4.4499) ├─┤ Rz(2.4406) ├┤ X ├┤ Ry(4.7037) ├»
         └────────────────┘└────────────┘ └────────────┘└───┘└────────────┘»
«         ┌─────────────┐     ┌────────────┐ ┌────────────┐     ┌────────────┐»
«qubit_0: ┤ Rz(0.87777) ├──■──┤ Ry(4.5983) ├─┤ Rz(1.4957) ├──■──┤ Ry(3.6191) ├»
«         └┬────────────┤┌─┴─┐├────────────┤┌┴────────────┤┌─┴─┐├────────────┤»
«qubit_1: ─┤ Rz(3.0728) ├┤ X ├┤ Ry(6.0018) ├┤ Rz(0.73668) ├┤ X ├┤ Ry(5.7226) ├»
«          └────────────┘└───┘└────────────┘└─────────────┘└───┘└────────────┘»
«         ┌───────────┐ 
«qubit_0: ┤ Rz(2.313) ├─
«         ├───────────┴┐
«qubit_1: ┤ Rz(5.6642) ├
«         └────────────┘
